In [ ]:
import pandas as pd
import numpy as np
import sys, os

import matplotlib.pyplot as plt
import json
import joblib

import geopandas as gdp

from datetime import datetime, timedelta

### Import Data

### Format data

In [ ]:
features = np.load('more_data.npy')

In [3]:
df = pd.DataFrame(features, columns=['ID', 'Longitude', 'Latitude', 'LL', 'Speed', 'Altitude', 'H_accuracy', 'V_accuracy', 'Timestamp'])

In [4]:
df = df.sort_values(['ID', 'Timestamp'])

In [5]:
df.Timestamp = df.Timestamp.str[:-3]

### Time Difference

In [6]:
df.Timestamp = pd.to_datetime(df.Timestamp)

In [11]:
df['TimeDiff'] = df.Timestamp.diff().dt.seconds + 1

In [12]:
df['SpeedDiff'] = df.Speed.diff()

In [13]:
df['Acceleration'] = df.SpeedDiff / df.TimeDiff

### Number of Unique Locations

In [15]:
df.ID = df.ID.astype(int)

In [16]:
x = (df.groupby('ID').LL.count() - df.groupby('ID').LL.nunique()).to_frame().reset_index()
x.columns = ['ID', 'DuplicationLL']
df = df.merge(x, on='ID', how='outer')

### Time of day

In [17]:
def rush_hours(x):
    if x >= 11 and x <= 13:
        return 'MR'
    elif x >= 20 and x <= 22:
        return 'NR'
    else:
        return 'S'

In [18]:
df['Hour'] = df.Timestamp.dt.hour
df['RushType'] = df.Timestamp.dt.hour.apply(rush_hours)

In [19]:
df.Timestamp.max(), df.Timestamp.min()

(Timestamp('2017-09-27 13:10:33'), Timestamp('2017-09-18 04:22:54'))

### PublicTransit isClosed

In [20]:
def public_transit_closed(x):
    if x >= 5 and x <= 9:
        return 1
    else:
        return 0

In [21]:
df['PTClosed'] = df.Timestamp.dt.hour.apply(public_transit_closed)

### Montreal Area

In [22]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lon1,lat1,lon2,lat2):
    # approximate radius of earth in km
    R = 6378.1

    lat_rad1 = radians(lat1)
    lon_rad1 = radians(lon1)
    lat_rad2 = radians(lat2)
    lon_rad2 = radians(lon2)

    dlon = lon_rad2 - lon_rad1
    dlat = lat_rad2 - lat_rad1

    a = sin(dlat / 2)**2 + cos(lat_rad1) * cos(lat_rad2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [23]:
cities = pd.read_csv('data/longlat.csv', header=None, names=['c1', 'c2', 'p1', 'p2'])

In [24]:
dists = {}
for index, row in zip(cities.index, cities.values):
    c1, c2, p1, p2 = row
    dist = calculate_distance(c1, c2, p1, p2)
    dists[index[0].lower()] = dist

In [25]:
centers = {}
for index, row in zip(cities.index, cities.values):
    c1, c2, _, _ = row
    centers[index[0].lower()] = (c1, c2)

In [26]:
centers

{'w': (45.458424, -73.837923),
 'd': (45.486655, -73.581342),
 'v': (45.527359000000004, -73.65242099999999),
 'e': (45.62249, -73.551357)}

In [27]:
def location(x):
    
    dist_w = calculate_distance(x[1], x[0], centers['w'][0], centers['w'][1])
    dist_d = calculate_distance(x[1], x[0], centers['d'][0], centers['d'][1])
    dist_v = calculate_distance(x[1], x[0], centers['v'][0], centers['v'][1])
    dist_e = calculate_distance(x[1], x[0], centers['e'][0], centers['e'][1])
    
    if dist_w <= dists['w']:
        return 'w'
    elif dist_d <= dists['d']:
        return 'd'
    elif dist_v <= dists['v']:
        return 'v'
    elif dist_e <= dists['e']:
        return 'e'
    else:
        return 'n'

In [28]:
data = []
for ll in df.LL.values:
    data.append(location(ll))
df['City'] = data

In [29]:
df.City.value_counts()

d    757725
v    510879
n    273793
e    263250
w    194353
Name: City, dtype: int64

### Distance

In [30]:
vals = [np.nan]
for i in range(1, df.shape[0]):
    
    long1, lat1 = df.Longitude[i-1], df.Latitude[i-1]
    long2, lat2 = df.Longitude[i], df.Latitude[i]
    
    vals.append(calculate_distance(long1, lat1, long2, lat2))

df['Distance'] = vals

In [31]:
df.head()

,ID,Longitude,Latitude,LL,Speed,Altitude,H_accuracy,V_accuracy,Timestamp,TimeDiff,SpeedDiff,Acceleration,DuplicationLL,Hour,RushType,PTClosed,City,Distance
0,2,-73.4885,45.5483,"(-73.488515, 45.548345)",15.2698,18.1288,8,3,2017-09-25 11:58:26,NaN,NaN,NaN,0,11,MR,0,n,NaN
1,2,-73.4891,45.548,"(-73.489116, 45.547977)",15.4348,17.9746,8,3,2017-09-25 11:58:30,5.0,0.164996,0.0329992,0,11,MR,0,n,0.062236
2,2,-73.4897,45.5476,"(-73.489714, 45.547606)",15.4219,18.3548,8,3,2017-09-25 11:58:34,5.0,-0.012937,-0.0025874,0,11,MR,0,n,0.062281
3,2,-73.4903,45.5472,"(-73.490302, 45.547239)",15.1126,18.1281,8,3,2017-09-25 11:58:38,5.0,-0.309246,-0.0618492,0,11,MR,0,n,0.061403
4,2,-73.4909,45.5468,"(-73.490863, 45.546836)",15.2363,17.973,6,3,2017-09-25 11:58:42,5.0,0.123711,0.0247422,0,11,MR,0,n,0.062652


### Bixie

In [32]:
bixie = pd.read_csv('data/Stations_2017.csv')

In [33]:
bixie.head()

,code,name,latitude,longitude,Unnamed: 4,Unnamed: 5
0,7015,LaSalle / 4e avenue,45.430740,-73.591911,NaN,NaN
1,6714,LaSalle / Senecal,45.434434,-73.586694,NaN,NaN
2,6712,LaSalle / Crawford,45.437914,-73.582740,NaN,NaN
3,6715,Natatorium (LaSalle / Rolland),45.444408,-73.575568,NaN,NaN
4,7048,Metro Angrignon,45.446534,-73.603541,NaN,NaN


In [34]:
start_end = df.groupby('ID').apply(lambda x: (x['LL'].values[0], x['LL'].values[-1])).to_frame().reset_index()
start_end.columns = ['ID', 'Coords']

In [35]:
start_end.head()

,ID,Coords
0,2,"((-73.488515, 45.548345), (-73.657023, 45.5501..."
1,19,"((-73.5201157013, 45.518476039), (-73.52241618..."
2,21,"((-73.640386, 45.550466), (-73.63895, 45.548075))"
3,33,"((-73.547435, 45.547262), (-73.55212, 45.545844))"
4,42,"((-73.603671, 45.546534), (-73.672126, 45.5473..."


In [36]:
results = []

for id_, coords in start_end.values:
    start = coords[0]
    end = coords[1]
    status = False
    
    for lat, long in zip(bixie.latitude, bixie.longitude):
        
        long1 = start[0]
        lat1 = start[1]
        
        start_dist = calculate_distance(long1, lat1, long, lat)
        
        long2 = end[0]
        lat2 = end[1]
        
        end_dist = calculate_distance(long2, lat2, long, lat)
        
        if start_dist <= 0.15 and end_dist <= 0.15:
            results.append([id_, 'bixie'])
            status = True
            break
            
    if not status:
        results.append([id_, 'no_bixie'])

In [37]:
bixies = pd.DataFrame(results, columns=['ID', 'Bixies'])

In [38]:
df = df.merge(bixies, on='ID', how='outer')

In [39]:
df.head()

,ID,Longitude,Latitude,LL,Speed,Altitude,H_accuracy,V_accuracy,Timestamp,TimeDiff,SpeedDiff,Acceleration,DuplicationLL,Hour,RushType,PTClosed,City,Distance,Bixies
0,2,-73.4885,45.5483,"(-73.488515, 45.548345)",15.2698,18.1288,8,3,2017-09-25 11:58:26,NaN,NaN,NaN,0,11,MR,0,n,NaN,no_bixie
1,2,-73.4891,45.548,"(-73.489116, 45.547977)",15.4348,17.9746,8,3,2017-09-25 11:58:30,5.0,0.164996,0.0329992,0,11,MR,0,n,0.062236,no_bixie
2,2,-73.4897,45.5476,"(-73.489714, 45.547606)",15.4219,18.3548,8,3,2017-09-25 11:58:34,5.0,-0.012937,-0.0025874,0,11,MR,0,n,0.062281,no_bixie
3,2,-73.4903,45.5472,"(-73.490302, 45.547239)",15.1126,18.1281,8,3,2017-09-25 11:58:38,5.0,-0.309246,-0.0618492,0,11,MR,0,n,0.061403,no_bixie
4,2,-73.4909,45.5468,"(-73.490863, 45.546836)",15.2363,17.973,6,3,2017-09-25 11:58:42,5.0,0.123711,0.0247422,0,11,MR,0,n,0.062652,no_bixie


### Weekday

In [40]:
df['Weekends'] = df.Timestamp.dt.day_name()

In [41]:
df.Weekends = df.Weekends.replace(to_replace=['Saturday', 'Sunday'], value=1)

In [42]:
df.Weekends = df.Weekends.replace(to_replace=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], value=0)

In [43]:
df.Weekends.value_counts()

0    1574858
1     425142
Name: Weekends, dtype: int64

### Time of Day

In [44]:
def time_of_day(x):
    if x >= 10 and x < 15:
        return 'morning'
    elif x >= 15 and x < 17:
        return 'lucnch'
    elif x >= 17 and x < 21:
        return 'afternoon'
    elif x >= 21 and x <= 23:
        return 'evening'
    elif x >= 0 and x < 10:
        return 'night'
    else:
        return np.nan

In [45]:
df['TimeOfDay'] = df.Hour.apply(time_of_day)

### Features

In [63]:
def pct(x):
    return np.quantile(x, 0.75)

In [64]:
df.ID = df.ID.astype(int)

In [67]:
df.Speed = df.Speed.astype(float)

In [68]:
temp = df.groupby('ID').agg({
    'Speed' : [
        ('SpeedPct', pct),
        ('MaxSpeed', 'max'),
        ('STDSpeed', 'std'),
        ('MeanSpeed', 'mean')
    ],
    'DuplicationLL' : ['first'],
    'Bixies' : ['first'],
    'Weekends' : ['first'],
    'RushType' : ['first'],
    'PTClosed' : ['first'],
    'City' : ['first']
})

In [69]:
def pct(x):
    if len(x) <= 1:
        return 0
    else:
        return np.quantile(x[1:], 0.75)

In [70]:
temp['AccelPct'] = df.groupby('ID').apply(lambda x: pct(x['Acceleration'].values))

In [71]:
def max_(x):
    if len(x) <= 1:
        return 0
    else:
        return max(x[1:])

In [72]:
temp['MaxAccel'] = df.groupby('ID').apply(lambda x: max_(x['Acceleration'].values))

In [73]:
def STD(x):
    if len(x) <= 1:
        return 0
    else:
        return np.std(x[1:])

In [74]:
temp['STDAccel'] = df.groupby('ID').apply(lambda x: STD(x['Acceleration'].values))

In [75]:
def mean(x):
    if len(x) <= 1:
        return 0
    else:
        return np.mean(x[1:])

In [76]:
temp['MeanAccel'] = df.groupby('ID').apply(lambda x: mean(x['Acceleration'].values))

In [77]:
def distance(x):
    if len(x) <= 1:
        return 0
    else:
        return np.sum(x[1:])

In [78]:
temp['TotalDistance'] = df.groupby('ID').apply(lambda x: distance(x['Distance'].values))

### Rename Columns

In [79]:
temp.head()

Speed                                DuplicationLL    Bixies Weekends  \
     SpeedPct   MaxSpeed  STDSpeed MeanSpeed         first     first    first   
ID                                                                              
2   13.179920  15.839291  5.095632  8.238649             0  no_bixie        0   
19   1.208492   1.281362  0.068257  1.163145             0  no_bixie        0   
21   1.670000   1.730000  0.145153  1.514667             0  no_bixie        0   
33   1.612500   2.800000  0.447859  1.408182             0  no_bixie        0   
42   5.750000   8.110000  1.419051  4.909139             0  no_bixie        1   

   RushType PTClosed  City  AccelPct  MaxAccel  STDAccel MeanAccel  \
      first    first first                                           
ID                                                                   
2        MR        0     n  0.094517  1.276043  0.365253 -0.027733   
19        S        0     e  0.001626  0.003101  0.002170 -0.000126   
21       NR        0     v  0.008554  0.020000  0.011845  0.000317   
33        S        0     e  0.009048  0.069412  0.022370  0.003218   
42        S        0     d  0.069643  0.488889  0.128816  0.006298   

   TotalDistance  
                  
ID                
2      16.385114  
19      0.367817  
21      0.358114  
33      0.537895  
42      7.524767

In [80]:
temp.columns = ['SpeedPct', 'MaxSpeed', 'STDSpeed', 'MeanSpeed', 'DuplcateLL', 'Bixies', 'isWeekend', 'RushType', 'PTClosed', 'City', 'AccelPct', 'MaxAccel', 'STDAccel', 'MeanAccel', 'TotalDistance']

In [81]:
temp.columns

Index(['SpeedPct', 'MaxSpeed', 'STDSpeed', 'MeanSpeed', 'DuplcateLL', 'Bixies',
       'isWeekend', 'RushType', 'PTClosed', 'City', 'AccelPct', 'MaxAccel',
       'STDAccel', 'MeanAccel', 'TotalDistance'],
      dtype='object')

In [82]:
temp.head()

,SpeedPct,MaxSpeed,STDSpeed,MeanSpeed,DuplcateLL,Bixies,isWeekend,RushType,PTClosed,City,AccelPct,MaxAccel,STDAccel,MeanAccel,TotalDistance
ID,,,,,,,,,,,,,,,
2,13.179920,15.839291,5.095632,8.238649,0,no_bixie,0,MR,0,n,0.094517,1.276043,0.365253,-0.027733,16.385114
19,1.208492,1.281362,0.068257,1.163145,0,no_bixie,0,S,0,e,0.001626,0.003101,0.002170,-0.000126,0.367817
21,1.670000,1.730000,0.145153,1.514667,0,no_bixie,0,NR,0,v,0.008554,0.020000,0.011845,0.000317,0.358114
33,1.612500,2.800000,0.447859,1.408182,0,no_bixie,0,S,0,e,0.009048,0.069412,0.022370,0.003218,0.537895
42,5.750000,8.110000,1.419051,4.909139,0,no_bixie,1,S,0,d,0.069643,0.488889,0.128816,0.006298,7.524767


In [83]:
temp.reset_index().to_csv('features.csv', index=False)

In [88]:
df[['ID', 'Longitude', 'Latitude']].to_csv('raw_data.csv', index=False)